In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras_tuner import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/KI_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # GRU layer 1
        x = layers.GRU(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        )(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # GRU layer 2
        x = layers.GRU(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )(x)
        x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'bilstm_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 02m 49s]
val_accuracy: 0.7968273162841797

Best val_accuracy So Far: 0.7968273162841797
Total elapsed time: 00h 27m 12s
Best Parameters for Week 1:
units_1: 256
dropout_1: 0.2
units_2: 192
dropout_2: 0.1
learning_rate: 0.0019513235178244249

Best Parameters for Week 2:
units_1: 96
dropout_1: 0.1
units_2: 96
dropout_2: 0.1
learning_rate: 0.0007634521198846196

Best Parameters for Week 3:
units_1: 96
dropout_1: 0.2
units_2: 160
dropout_2: 0.2
learning_rate: 0.0019833455634702653

Best Parameters for Week 4:
units_1: 128
dropout_1: 0.5
units_2: 256
dropout_2: 0.2
learning_rate: 0.0010692731625104286


In [7]:
# Sau khi tìm ra best_params_week3, tiến hành build và train mô hình với tham số tối ưu
def build_best_gru_model(best_params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    x = layers.GRU(units=best_params.get('units_1'), return_sequences=True)(inputs)
    x = layers.Dropout(rate=best_params.get('dropout_1'))(x)
    
    x = layers.GRU(units=best_params.get('units_2'), return_sequences=False)(x)
    x = layers.Dropout(rate=best_params.get('dropout_2'))(x)
    
    outputs = layers.Dense(5, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params.get('learning_rate')),
                  loss=focal_loss(gamma=2., alpha=0.25),
                  metrics=['accuracy'])
    return model


# Chuẩn bị dữ liệu giống như trước cho tuần 3
train_data = pd.read_csv(file_paths_train["week3"])
val_data = pd.read_csv(file_validation["week3"])

X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
y_train = train_data["classification_encoded"]

X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
y_val = val_data["classification_encoded"]

oversampler = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)

y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

# Xây dựng mô hình với tham số tốt nhất
model_week3 = build_best_gru_model(best_params_week3, input_shape=(X_train_res.shape[1], 1))

# Train mô hình
model_week3.fit(X_train_res, y_train_res,
                validation_data=(X_val, y_val),
                epochs=20,
                batch_size=32)

# ✅ Lưu mô hình dưới dạng .h5
model_week3.save("gru_week3_model.h5")

print("✅ Đã lưu mô hình GRU tuần 3 dưới dạng gru_week3_model.h5")

Epoch 1/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.3624 - loss: 0.2249 - val_accuracy: 0.6565 - val_loss: 0.1440
Epoch 2/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.6084 - loss: 0.1559 - val_accuracy: 0.7023 - val_loss: 0.1225
Epoch 3/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.6487 - loss: 0.1381 - val_accuracy: 0.6595 - val_loss: 0.1333
Epoch 4/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.6680 - loss: 0.1277 - val_accuracy: 0.7419 - val_loss: 0.1066
Epoch 5/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.6937 - loss: 0.1146 - val_accuracy: 0.7425 - val_loss: 0.1039
Epoch 6/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7142 - loss: 0.1039 - val_accuracy: 0.7535 - val_loss: 0.1009
Epoch 7/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7361 - loss: 0.0953 - val_accuracy: 0.7718 - val_loss: 0.0985
Epoch 8/20
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7519 - loss: 0.0883 - 

## Danh sách tham số tốt nhất của từng tuần

In [8]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [9]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình BiLSTM
def build_GRU_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)  # Định nghĩa đầu vào
    
    # GRU layer 1
    x = layers.GRU(
        units=params.get('units_1'),
        return_sequences=True
    )(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # GRU layer 2
    x = layers.GRU(
        units=params.get('units_2', 32),
        return_sequences=False
    )(x)
    x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    
    return model


In [10]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        input_shape = (X_train.shape[1], X_train.shape[2])
        model = build_GRU_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 256, 'dropout_1': 0.2, 'units_2': 192, 'dropout_2': 0.1, 'learning_rate': 0.0019513235178244249}
Fold 1: Using file /kaggle/input/KI_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6300 - loss: 0.1357 - val_accuracy: 0.6763 - val_loss: 0.1145
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6712 - loss: 0.1191 - val_accuracy: 0.7064 - val_loss: 0.1110
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6849 - loss: 0.1149 - val_accuracy: 0.6778 - val_loss: 0.1098
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6756 - loss: 0.1142 - val_accuracy: 0.6984 - val_loss: 0.1075
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6880 - loss: 0.1120 - val_accuracy: 0.7259 - val_loss: 0.1048
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6923 - los

## Kết quả cross validation trên 5-folds

In [11]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.7792855501174927
Average Train Time: 85.5700 seconds
Average Test Time: 0.4399 seconds
Average AUC Macro: 0.8816287031803715
Average AUC Weighted: 0.8937927384391487

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.629935        0.716333          0.669509     0.877962
1      1           0.871448        0.116536          0.199598     0.872404
2      2           0.660003        0.468426          0.544475     0.862229
3      3           0.632116        0.497006          0.543288     0.867065
4      4           0.843558        0.879377          0.860779     0.894302

Average Confusion Matrix:
       0     1     2     3       4
0  429.8   0.4  11.8   8.4   149.6
1   52.2  10.2   2.0   2.8    20.4
2   49.0   0.2  77.0   3.6    34.6
3   23.6   0.0   2.2  83.0    58.2
4  129.0   1.6  24.6  38.2  1410.0

=== Results for week2 ===
Average Accuran

## Kiểm tra trên tập test

In [12]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])
    model = build_GRU_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    # Tính các chỉ số Precision, Recall, F1 cho từng lớp và macro
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [13]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4734 - loss: 0.1983 - val_accuracy: 0.6730 - val_loss: 0.2273
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5407 - loss: 0.1757 - val_accuracy: 0.6301 - val_loss: 0.2356
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5749 - loss: 0.1647 - val_accuracy: 0.7146 - val_loss: 0.2099
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5947 - loss: 0.1592 - val_accuracy: 0.6533 - val_loss: 0.2149
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6092 - loss: 0.1515 - val_accuracy: 0.7131 - val_loss: 0.2062
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6200 - loss: 0.1472 - val_accuracy: 0.6914 - val_loss: 0.1951
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6205 - loss: 0.1453 - val_accuracy: 0.6645 - val_loss: 0.2107
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [14]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4863 - loss: 0.1958 - val_accuracy: 0.4645 - val_loss: 0.2543
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5829 - loss: 0.1656 - val_accuracy: 0.6785 - val_loss: 0.1892
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6040 - loss: 0.1557 - val_accuracy: 0.6383 - val_loss: 0.2168
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6209 - loss: 0.1487 - val_accuracy: 0.6690 - val_loss: 0.2093
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6201 - loss: 0.1479 - val_accuracy: 0.6855 - val_loss: 0.2237
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6333 - loss: 0.1418 - val_accuracy: 0.7219 - val_loss: 0.1904
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6384 - loss: 0.1377 - val_accuracy: 0.6530 - val_loss: 0.1801
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [15]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.4652 - loss: 0.1993 - val_accuracy: 0.5266 - val_loss: 0.2608
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5626 - loss: 0.1690 - val_accuracy: 0.7131 - val_loss: 0.2214
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5880 - loss: 0.1629 - val_accuracy: 0.6505 - val_loss: 0.2272
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6082 - loss: 0.1546 - val_accuracy: 0.7002 - val_loss: 0.1967
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6143 - loss: 0.1515 - val_accuracy: 0.6835 - val_loss: 0.1827
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6192 - loss: 0.1489 - val_accuracy: 0.6633 - val_loss: 0.2198
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6268 - loss: 0.1448 - val_accuracy: 0.6311 - val_loss: 0.2198
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [16]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4537 - loss: 0.2036 - val_accuracy: 0.5982 - val_loss: 0.2554
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5487 - loss: 0.1756 - val_accuracy: 0.6588 - val_loss: 0.2060
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5903 - loss: 0.1628 - val_accuracy: 0.6795 - val_loss: 0.2322
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6025 - loss: 0.1588 - val_accuracy: 0.6543 - val_loss: 0.2380
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6150 - loss: 0.1533 - val_accuracy: 0.6645 - val_loss: 0.2445
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6150 - loss: 0.1512 - val_accuracy: 0.6061 - val_loss: 0.2123
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6137 - loss: 0.1496 - val_accuracy: 0.6308 - val_loss: 0.2315
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [17]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 270.43 seconds
  Test Time: 0.39 seconds
  Accurancy: 0.7292682926829268
  Precision: [0.7275641  0.22527473 0.45801527 0.49193548 0.90572391]
  Recall: [0.60533333 0.75925926 0.58252427 0.58095238 0.80458624]
  F1-Score: [0.66084425 0.34745763 0.51282051 0.53275109 0.85216473]
  Macro Precision: 0.5617026969218548
  Macro Recall: 0.6665310973331611
  Macro F1-Score: 0.5812076425469462
  Confusion Matrix:
[[227  56  30   6  56]
 [  5  41   0   1   7]
 [ 16  16  60   4   7]
 [  8  11  11  61  14]
 [ 56  58  30  52 807]]

=== AUC-ROC ===
AUC Macro: 0.8864
AUC Weighted: 0.8949
AUC per Label: [0.88313963 0.88892651 0.87743109 0.87949681 0.90295643]
Week 2:
  Train Time: 267.42 seconds
  Test Time: 0.38 seconds
  Accurancy: 0.7774390243902439
  Precision: [0.78338279 0.24832215 0.5412844  0.62727273 0.90481283]
  Recall: [0.704      0.68518519 0.57281553 0.65714286 0.84346959]
  F1-Score: [0.74157303 0.36453202 0.55660377 0.641860